##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Políticas

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/3_policies_tutorial">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/3_policies_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/3_policies_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/agents/tutorials/3_policies_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Introdução

Na terminologia de Aprendizado por Reforço, as políticas mapeiam uma observação do ambiente a uma ação ou a uma distribuição sobre ações. No TF-Agents, as observações do ambiente são contidas em uma tupla nomeada `TimeStep('step_type', 'discount', 'reward', 'observation')`, e as políticas mapeiam timesteps a ações ou distribuições sobre ações. A maioria das políticas usam `timestep.observation`, algumas usam `timestep.step_type` (por exemplo, para redefinir o estado no início de um episódio em políticas stateful), mas `timestep.discount` e `timestep.reward` são geralmente ignorados.

As políticas são relacionadas a outros componentes no TF-Agents da seguinte maneira. A maioria das políticas tem uma rede neural para computar ações e/ou distribuições sobre ações de TimeSteps. Os agentes podem conter uma ou mais políticas para diferentes fins, por exemplo, uma política principal treinada para implantação e uma política ruidosa para coleta de dados. As políticas podem ser salvas/restauradas e usadas independentemente do agente para coleta de dados, avaliação etc.

Algumas políticas são mais fáceis de escrever no TensorFlow (por exemplo, aquelas com uma rede neural), enquanto outras são mais fáceis de escrever no Python (por exemplo, seguindo um script de ações). Portanto, no TF agents, permitimos ambas as políticas do Python e do TensorFlow. Além disso, as políticas escritas no TensorFlow talvez precisem ser usadas em um ambiente do Python ou vice-versa. Por exemplo, uma política do TensorFlow é usada para treinamento, mas depois é implantada em um ambiente de produção do Python. Para facilitar isso, fornecemos wrappers para a conversão entre políticas do Python e do TensorFlow.

Outra classe interessante de políticas são os wrappers de políticas, que modificam uma determinada política de uma maneira específica. Por exemplo, adicionam um tipo especial de ruído, criam uma versão greedy ou epsilon-greedy de uma política estocástica, misturam várias políticas de maneira aleatória etc. 

## Configuração

Se você ainda não instalou o tf-agents, execute:

In [ ]:
!pip install tf-agents

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

from tf_agents.specs import array_spec
from tf_agents.specs import tensor_spec
from tf_agents.networks import network

from tf_agents.policies import py_policy
from tf_agents.policies import random_py_policy
from tf_agents.policies import scripted_py_policy

from tf_agents.policies import tf_policy
from tf_agents.policies import random_tf_policy
from tf_agents.policies import actor_policy
from tf_agents.policies import q_policy
from tf_agents.policies import greedy_policy

from tf_agents.trajectories import time_step as ts

## Políticas do Python

A interface para as políticas do Python é definida em `policies/py_policy.PyPolicy`. Os principais métodos são:


In [ ]:
class Base(object):

  @abc.abstractmethod
  def __init__(self, time_step_spec, action_spec, policy_state_spec=()):
    self._time_step_spec = time_step_spec
    self._action_spec = action_spec
    self._policy_state_spec = policy_state_spec

  @abc.abstractmethod
  def reset(self, policy_state=()):
    # return initial_policy_state.
    pass

  @abc.abstractmethod
  def action(self, time_step, policy_state=()):
    # return a PolicyStep(action, state, info) named tuple.
    pass

  @abc.abstractmethod
  def distribution(self, time_step, policy_state=()):
    # Not implemented in python, only for TF policies.
    pass

  @abc.abstractmethod
  def update(self, policy):
    # update self to be similar to the input `policy`.
    pass

  @property
  def time_step_spec(self):
    return self._time_step_spec

  @property
  def action_spec(self):
    return self._action_spec

  @property
  def policy_state_spec(self):
    return self._policy_state_spec

O método mais importante é `action(time_step)`, que mapeia um `time_step` com uma observação do ambiente para uma tupla nomeada PolicyStep que contém os seguintes atributos:

- `action`: a ação a ser aplicada ao ambiente.
- `state`: o estado de uma política (por exemplo, estado RNN) a ser alimentado na próxima chamada de ação.
- `info`: informações extras opcionais, como probabilidades log da ação.

O `time_step_spec` e `action_spec` são especificações para o timestep de entrada e a ação de saída. As políticas também têm uma função `reset`, que é geralmente usada para redefinir o estado em políticas stateful. A função `update(new_policy)` atualiza `self` em relação a `new_policy`.

Agora, vamos analisar alguns exemplos de políticas do Python.


### Exemplo 1: política aleatória do Python

Um exemplo simples de uma `PyPolicy` é a `RandomPyPolicy`, que gera ações aleatórias para a determinada action_spec discreta/contínua. A entrada `time_step` é ignorada.

In [ ]:
action_spec = array_spec.BoundedArraySpec((2,), np.int32, -10, 10)
my_random_py_policy = random_py_policy.RandomPyPolicy(time_step_spec=None,
    action_spec=action_spec)
time_step = None
action_step = my_random_py_policy.action(time_step)
print(action_step)
action_step = my_random_py_policy.action(time_step)
print(action_step)

### Exemplo 2: política do Python com script

Uma política com script reproduz um script de ações representadas como uma lista de tuplas `(num_repeats, action)`. Sempre que a função `action` é chamada, ela retorna a próxima ação da lista até que o número especificado de repetições seja atingido e, então, segue para a próxima ação da lista. O método `reset` pode ser chamado para começar a execução desde o início da lista.

In [ ]:
action_spec = array_spec.BoundedArraySpec((2,), np.int32, -10, 10)
action_script = [(1, np.array([5, 2], dtype=np.int32)), 
                 (0, np.array([0, 0], dtype=np.int32)), # Setting `num_repeats` to 0 will skip this action.
                 (2, np.array([1, 2], dtype=np.int32)), 
                 (1, np.array([3, 4], dtype=np.int32))]

my_scripted_py_policy = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=None, action_spec=action_spec, action_script=action_script)

policy_state = my_scripted_py_policy.get_initial_state()
time_step = None
print('Executing scripted policy...')
action_step = my_scripted_py_policy.action(time_step, policy_state)
print(action_step)
action_step= my_scripted_py_policy.action(time_step, action_step.state)
print(action_step)
action_step = my_scripted_py_policy.action(time_step, action_step.state)
print(action_step)

print('Resetting my_scripted_py_policy...')
policy_state = my_scripted_py_policy.get_initial_state()
action_step = my_scripted_py_policy.action(time_step, policy_state)
print(action_step)

## Políticas do TensorFlow

As políticas do TensorFlow seguem a mesma interface que as políticas do Python. Vamos conferir alguns exemplos.

### Exemplo 1: Random TF Policy

Uma RandomTFPolicy pode ser usada para gerar ações aleatórias de acordo com uma `action_spec` discreta/contínua. A entrada `time_step` é ignorada.


In [ ]:
action_spec = tensor_spec.BoundedTensorSpec(
    (2,), tf.float32, minimum=-1, maximum=3)
input_tensor_spec = tensor_spec.TensorSpec((2,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)

my_random_tf_policy = random_tf_policy.RandomTFPolicy(
    action_spec=action_spec, time_step_spec=time_step_spec)
observation = tf.ones(time_step_spec.observation.shape)
time_step = ts.restart(observation)
action_step = my_random_tf_policy.action(time_step)

print('Action:')
print(action_step.action)

### Exemplo 2: política de ator

Uma política de ator pode ser criada usando uma rede que mapeia `time_steps` a ações ou uma rede que mapeia `time_steps` a distribuições sobre ações.


#### Usando uma rede de ações

Vamos definir uma rede da seguinte maneira:

In [ ]:
class ActionNet(network.Network):

  def __init__(self, input_tensor_spec, output_tensor_spec):
    super(ActionNet, self).__init__(
        input_tensor_spec=input_tensor_spec,
        state_spec=(),
        name='ActionNet')
    self._output_tensor_spec = output_tensor_spec
    self._sub_layers = [
        tf.keras.layers.Dense(
            action_spec.shape.num_elements(), activation=tf.nn.tanh),
    ]

  def call(self, observations, step_type, network_state):
    del step_type

    output = tf.cast(observations, dtype=tf.float32)
    for layer in self._sub_layers:
      output = layer(output)
    actions = tf.reshape(output, [-1] + self._output_tensor_spec.shape.as_list())

    # Scale and shift actions to the correct range if necessary.
    return actions, network_state

No TensorFlow, a maioria das camadas de rede são criadas para operações em lote, então esperamos que os time_steps de entrada sejam divididos em lotes, e a saída da rede também será dividida em lotes. Além disso, a rede é responsável por produzir ações no intervalo correto para a action_spec específica. Convencionalmente, isso é feito usando, por exemplo, uma ativação tanh para a camada final produzir ações em [-1, 1]. Em seguida, escalando e mudando isso para o intervalo correto como a action_spec de entrada (por exemplo, veja `tf_agents/agents/ddpg/networks.actor_network()`).

Agora, criamos uma política de ator usando a rede acima.

In [ ]:
input_tensor_spec = tensor_spec.TensorSpec((4,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)
action_spec = tensor_spec.BoundedTensorSpec((3,),
                                            tf.float32,
                                            minimum=-1,
                                            maximum=1)

action_net = ActionNet(input_tensor_spec, action_spec)

my_actor_policy = actor_policy.ActorPolicy(
    time_step_spec=time_step_spec,
    action_spec=action_spec,
    actor_network=action_net)

Podemos aplicar isso a qualquer lote de time_steps que seguem time_step_spec:

In [ ]:
batch_size = 2
observations = tf.ones([2] + time_step_spec.observation.shape.as_list())

time_step = ts.restart(observations, batch_size)

action_step = my_actor_policy.action(time_step)
print('Action:')
print(action_step.action)

distribution_step = my_actor_policy.distribution(time_step)
print('Action distribution:')
print(distribution_step.action)

No exemplo acima, criamos a política usando uma rede de ações que produz um tensor de ação. Nesse caso, `policy.distribution(time_step)` é uma distribuição determinística (delta) em torno da saída de `policy.action(time_step)`. Uma forma de produzir uma política estocástica é envolver a política de ator em um wrapper de política que adiciona ruído às ações. Outra forma é criar a política de ator usando uma rede de distribuição de ações em vez de uma rede de ações, conforme mostrado abaixo.

#### Usando uma rede de distribuição de ações

In [ ]:
class ActionDistributionNet(ActionNet):

  def call(self, observations, step_type, network_state):
    action_means, network_state = super(ActionDistributionNet, self).call(
        observations, step_type, network_state)

    action_std = tf.ones_like(action_means)
    return tfp.distributions.MultivariateNormalDiag(action_means, action_std), network_state


action_distribution_net = ActionDistributionNet(input_tensor_spec, action_spec)

my_actor_policy = actor_policy.ActorPolicy(
    time_step_spec=time_step_spec,
    action_spec=action_spec,
    actor_network=action_distribution_net)

action_step = my_actor_policy.action(time_step)
print('Action:')
print(action_step.action)
distribution_step = my_actor_policy.distribution(time_step)
print('Action distribution:')
print(distribution_step.action)

Observe que, acima, as ações são cortadas para ficarem no intervalo da especificação da determinada ação [-1, 1]. Isso é devido a um argumento de construtor de ActorPolicy clip=True por padrão. A definição dele como "false" retornará ações não cortadas produzidas pela rede. 

As políticas estocásticas podem ser convertidas em políticas determinísticas usando, por exemplo, um wrapper GreedyPolicy que escolhe `stochastic_policy.distribution().mode()` como sua ação, e uma distribuição determinística/delta em torno dessa ação greedy como sua `distribution()`.

### Exemplo 3: política Q

Uma política Q é usada em agentes como DQN e é baseada em uma rede Q que prevê um valor Q para cada ação discreta. Para um timestep específico, a ação de distribuição na Política Q é uma distribuição categórica criada usando os valores q como logits.


In [ ]:
input_tensor_spec = tensor_spec.TensorSpec((4,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)
action_spec = tensor_spec.BoundedTensorSpec((),
                                            tf.int32,
                                            minimum=0,
                                            maximum=2)
num_actions = action_spec.maximum - action_spec.minimum + 1


class QNetwork(network.Network):

  def __init__(self, input_tensor_spec, action_spec, num_actions=num_actions, name=None):
    super(QNetwork, self).__init__(
        input_tensor_spec=input_tensor_spec,
        state_spec=(),
        name=name)
    self._sub_layers = [
        tf.keras.layers.Dense(num_actions),
    ]

  def call(self, inputs, step_type=None, network_state=()):
    del step_type
    inputs = tf.cast(inputs, tf.float32)
    for layer in self._sub_layers:
      inputs = layer(inputs)
    return inputs, network_state


batch_size = 2
observation = tf.ones([batch_size] + time_step_spec.observation.shape.as_list())
time_steps = ts.restart(observation, batch_size=batch_size)

my_q_network = QNetwork(
    input_tensor_spec=input_tensor_spec,
    action_spec=action_spec)
my_q_policy = q_policy.QPolicy(
    time_step_spec, action_spec, q_network=my_q_network)
action_step = my_q_policy.action(time_steps)
distribution_step = my_q_policy.distribution(time_steps)

print('Action:')
print(action_step.action)

print('Action distribution:')
print(distribution_step.action)

## Wrappers de políticas

Um wrapper de política pode ser usado para envolver e modificar uma política específica, por exemplo, adicionar ruído. Os wrappers de política são uma subclasse de Policy (Python/TensorFlow) e, portanto, podem ser usados como qualquer outra política. 

### Exemplo: política greedy

Um wrapper greedy pode ser usado para envolver qualquer política do TensorFlow que implemente `distribution()`. `GreedyPolicy.action()` retornará `wrapped_policy.distribution().mode()` e `GreedyPolicy.distribution()` é uma distribuição determinística/delta em torno de `GreedyPolicy.action()`:

In [ ]:
my_greedy_policy = greedy_policy.GreedyPolicy(my_q_policy)

action_step = my_greedy_policy.action(time_steps)
print('Action:')
print(action_step.action)

distribution_step = my_greedy_policy.distribution(time_steps)
print('Action distribution:')
print(distribution_step.action)